# Exercise 2: Bumeran Webscrapping

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
for pagina in range(1, 7):
    if pagina == 1:
        url = base
    else:
        url = f"{base}?page={pagina}"

In [9]:
urlBase = "https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html"


driver = webdriver.Chrome()
driver.maximize_window()     
driver.get(url)

In [19]:
def scrappearPagina(driver, urlBase, pagina, ANUNCIOS):
    url = urlBase + "?page=" + pagina

    driver.get(url)
    time.sleep(3)

    wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '/empleos/')]")))

    # Extraer links
    links = driver.find_elements(By.XPATH, "//a[contains(@href, '/empleos/')]")
    urls = [link.get_attribute("href") for link in links if link.get_attribute("href")]
    urls_filtradas = list(set([u for u in urls if "bumeran.com.pe/empleos/" in u]))

    print(f"🔗 Enlaces encontrados en página {pagina}: {len(urls_filtradas)}")
    ANUNCIOS.update(urls_filtradas)

    return ANUNCIOS



In [20]:
ANUNCIOS = set()
aa = scrappearPagina(driver, urlBase, 1, ANUNCIOS)


TypeError: can only concatenate str (not "int") to str

In [ ]:
aa

In [ ]:


# Inicializar
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
todas_las_urls = set()

# Recorremos las 6 páginas
for pagina in range(1, 7):
    if pagina == 1:
        url = base
    else:
        url = f"{base}?page={pagina}"

    print(f"\n🌐 Visitando página {pagina}...")
    driver.get(url)
    time.sleep(3)

    try:
        # Esperar a que cargue algún enlace
        wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[contains(@href, '/empleos/')]")))

        # Extraer enlaces
        enlaces = driver.find_elements(By.XPATH, "//a[contains(@href, '/empleos/')]")
        urls = [e.get_attribute("href") for e in enlaces if e.get_attribute("href")]
        urls_filtradas = list(set([u for u in urls if "bumeran.com.pe/empleos/" in u]))

        print(f"🔗 Enlaces encontrados en página {pagina}: {len(urls_filtradas)}")
        todas_las_urls.update(urls_filtradas)

    except Exception as e:
        print(f"❌ Error en página {pagina}: {e}")

driver.quit()

# Resultados finales
print("\n✅ Total de enlaces únicos encontrados:", len(todas_las_urls))
for url in todas_las_urls:
    print(url)



🌐 Visitando página 1...
🔗 Enlaces encontrados en página 1: 20

🌐 Visitando página 2...
🔗 Enlaces encontrados en página 2: 20

🌐 Visitando página 3...
🔗 Enlaces encontrados en página 3: 20

🌐 Visitando página 4...
🔗 Enlaces encontrados en página 4: 20

🌐 Visitando página 5...
🔗 Enlaces encontrados en página 5: 20

🌐 Visitando página 6...
🔗 Enlaces encontrados en página 6: 6

✅ Total de enlaces únicos encontrados: 106
https://www.bumeran.com.pe/empleos/desarrollador-full-stack-senior--ecommerce-magento-perfumerias-unidas-1117965886.html
https://www.bumeran.com.pe/empleos/analista-de-programacion-rpg-izipay-1117080379.html
https://www.bumeran.com.pe/empleos/desarrollador-full-stack-surco-presencial-well-consulting-services-1117978454.html
https://www.bumeran.com.pe/empleos/analista-programador-java-1117972678.html
https://www.bumeran.com.pe/empleos/desarrolladora-mineria-lurin-zamine-peru-1117976772.html
https://www.bumeran.com.pe/empleos/soporte-jr-de-aplicaciones-linux-plsql-hitss-peru

In [127]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# === U API KEY ===
API_KEY = "31b48495682ce08133cfcb96b70c403b"

# === FUNCIÓN PARA PEDIR PÁGINAS CON SCRAPERAPI ===
def get_page(url):
    full_url = f"http://api.scraperapi.com?api_key={API_KEY}&url={url}&render=true"
    response = requests.get(full_url)
    response.raise_for_status()
    return response.text

# === FUNCIÓN PARA CORTAR DESCRIPCIÓN HASTA "BENEFICIOS" ===
def cortar_descripcion(texto):
    for palabra in ["beneficios", "BENEFICIOS", "Beneficios", "REQUISITOS"]:
        idx = texto.lower().find(palabra.lower())
        if idx != -1:
            return texto[:idx]
    return texto

# === PEGA AQUÍ TUS 109 LINKS COMPLETOS ===
urls = [
    "https://www.bumeran.com.pe/empleos/tech-lead-lider-tecnico-frontend-react-nextjs-zutun.-1117959935.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-backend-inetum-peru-1117968206.html",
    "https://www.bumeran.com.pe/empleos/programador-bms-indra-peru-1117973363.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-java-senior-azure-temporal-indra-peru-1117978869.html",
    "https://www.bumeran.com.pe/empleos/programador-rpa-rocketbot-uipath-valtx-1117968630.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-full-stack-grupo-red-salud-1117971064.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-python-odoo-ibr-peru-s.a.-1117961539.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-sector-educativo-inlearning-escuelas-de-educacion-superior-1117973244.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-full-stack-senior--ecommerce-magento-perfumerias-unidas-1117965886.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-full-stack-universidad-de-ingenieria-y-tecnologia-utec-1117961694.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-backend-nodejs-aws-indra-peru-1117974651.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-frontend-senior-fractal-soluciones-it-1117974205.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-backend-javascript-senior-remoto-csti-corp-1117970346.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-java-microservicios-canvia-1117960644.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-genexus-remoto-software-enterprise-services-s.a.c.-1117980075.html",
    "https://www.bumeran.com.pe/empleos/programador-1117960615.html",
    "https://www.bumeran.com.pe/empleos/analista-de-programacion-caja-metropolitana-de-lima-1117975863.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-frontend-senior-canvia-1117964197.html",
    "https://www.bumeran.com.pe/empleos/programador-bms-o-scada-indra-peru-1117975855.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-fullstack-lenguaje-go-y-react-native-software-enterprise-services-s.a.c.-1117949402.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-java-valtx-1117962716.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-java-cloud-senior-green-solutions-1117971404.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-php-canvia-1117976775.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-full-stack-igts-sac-1117965145.html",
    "https://www.bumeran.com.pe/empleos/programador-python-junior-csti-corp-1117970174.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-backend-semi-senior-metrica-peru-1117979121.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-de-sistemas-intelitech-peru-s.a.c.-1117964390.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-postgre-oracle-hibrido-green-solutions-1117962598.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-postgre-oracle-hibrido-green-solutions-1117979984.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-java-remoto-planilla-completa-green-solutions-1117971817.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-backend-nodejs-senior-zoluxiones-sac-1117977115.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-postgre-oracle-hibrido-green-solutions-1117970409.html",
    "https://www.bumeran.com.pe/empleos/analista-de-programador-derrama-magisterial-1117974631.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-sede-ate-bumeran-selecta-1117969140.html",
    "https://www.bumeran.com.pe/empleos/practicante-pre-profesional-de-automatizacion-canvia-1117980437.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-backend-javascript-senior-csti-corp-1117961392.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-fullstack-java-hitss-peru-1117980080.html",
    "https://www.bumeran.com.pe/empleos/programador-de-inteligencia-artificial-growth-digital-lab-1117968752.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-fullstack-1117976482.html",
    "https://www.bumeran.com.pe/empleos/desarrolladora-mineria-lurin-zamine-peru-1117976772.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-full-stack-node.js-c-java-angularjs-ibr-peru-s.a.-1117967655.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-frontend-angular-canvia-1117960612.html",
    "https://www.bumeran.com.pe/empleos/programador-electrico-polindustria-s.a-1117979364.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-backend-indra-peru-1117968754.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-fullstack-isp-consulting-1117960954.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-.net-%7C-visual-basic-entelgy-1117975151.html",
    "https://www.bumeran.com.pe/empleos/practicante-profesional-desarrollador-pl-sql-indra-peru-1117941356.html",
    "https://www.bumeran.com.pe/empleos/soporte-jr-de-aplicaciones-linux-plsql-hitss-peru-1117963093.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-full-stack-.net-ibr-peru-s.a.-1117964229.html", "https://www.bumeran.com.pe/empleos/practicante-profesional-programacion-plsql-indra-peru-1117960022.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-backend-.net-indra-peru-1117971369.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-1117964091.html",
    "https://www.bumeran.com.pe/empleos/programador-bms-automizador-indra-peru-1117973373.html",
    "https://www.bumeran.com.pe/empleos/backend-developer-senior-culqi-talent2win-1117963319.html",
    "https://www.bumeran.com.pe/empleos/fullstack-developer-senior-.net-angular-zutun.-1117971550.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-.net-senior-fractal-soluciones-it-1117973977.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-lider-tecnico-protiviti-peru-1117978985.html",
    "https://www.bumeran.com.pe/empleos/analista-desarrollador-java-sector-administracion-de-fondos-colectivos-experis-peru-1117978810.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-backend-manpowergroup-peru-1117964291.html",
    "https://www.bumeran.com.pe/empleos/trainee-programador-jr-building-software-1117968407.html",
    "https://www.bumeran.com.pe/empleos/programador-en-oracle-developer-clbs-1117972036.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-full-stack-surco-presencial-well-consulting-services-1117978454.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-full-stack-java-aws-react-native-software-enterprise-services-s.a.c.-1117974521.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-java-mobile-red-educativa-le-cordon-bleu-1117979044.html",
    "https://www.bumeran.com.pe/empleos/analista-desarrollador-full-stack-gruporpp-s.a.c.-1117968336.html",
    "https://www.bumeran.com.pe/empleos/software-engineer-senior-grupo-gloria-1117976663.html",
    "https://www.bumeran.com.pe/empleos/senior-react-developer-encora-1117979242.html",
    "https://www.bumeran.com.pe/empleos/programador-full-stack-c-exp-en-ia-valtx-1117973381.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-senior-adex-1117972522.html",
    "https://www.bumeran.com.pe/empleos/analista-desarrollador-java-manpowergroup-peru-1117974662.html",
    "https://www.bumeran.com.pe/empleos/programador-bms-indra-peru-1117974062.html",
    "https://www.bumeran.com.pe/empleos/programador-full-stack-java-react-valtx-1117977502.html",
    "https://www.bumeran.com.pe/empleos/programador-ologgi-s.a.c-1117973857.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-.net-php-indra-peru-1117961552.html",
    "https://www.bumeran.com.pe/empleos/analista-senior-de-automatizaciones-e-inteligencia-artificial-hibrido-cibergestion-peru-1117964801.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-fullstack-semiseniorjava-angular-csti-corp-1117961713.html",
    "https://www.bumeran.com.pe/empleos/analista-de-imagenes-hospitalarias-sistemas-grupo-san-pablo-1117977289.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-java-entelgy-1117975157.html",
    "https://www.bumeran.com.pe/empleos/mid-node-aws-engineer-encora-1117969139.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-.net-sqlserver-integration-quality-project-management-s.a.c.-iq-project-s.a.c.-1117979719.html",
    "https://www.bumeran.com.pe/empleos/practicante-de-digitalizacion-repsol-1117960947.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-frontend-react-por-proyecto-indra-peru-1117965904.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-fullstack-java-hitss-peru-1117967606.html",
    "https://www.bumeran.com.pe/empleos/frontend-developer-angular-experis-peru-1117962336.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-clinica-vesalio-s.a.-1117966201.html",
    "https://www.bumeran.com.pe/empleos/programador-bms-indra-peru-1117973221.html",
    "https://www.bumeran.com.pe/empleos/soporte-jr-y-sr-de-plataforma-bmc-helix-remedy-hitss-peru-1117968539.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-de-ti-seaboard-overseas-peru-s.a.-1117970924.html",
    "https://www.bumeran.com.pe/empleos/java-developer-senior-inetum-peru-1117979249.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-genexus-software-enterprise-services-s.a.c.-1117965079.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-java-oracle-presencial-hitss-peru-1117963095.html",
    "https://www.bumeran.com.pe/empleos/auditor-en-tasador-de-joyas-caja-metropolitana-de-lima-1117977845.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-visual-basic-.net-idm-technology-1117979310.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-de-integracion-osb-stefanini-peru-s.a-1117978939.html",
    "https://www.bumeran.com.pe/empleos/tecnico-desarrollador-de-sistemas-1117973188.html",
    "https://www.bumeran.com.pe/empleos/analista-de-programacion-rpg-izipay-1117080379.html",
    "https://www.bumeran.com.pe/empleos/.net-developer-junior-inetum-peru-1117970031.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-mobile-react-native-fractal-soluciones-it-1117974334.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-fullsatck-enfoque-en-frontend-fractal-soluciones-it-1117962666.html", "https://www.bumeran.com.pe/empleos/analista-programador-vb6-real-systems-s-a-1117886662.html",
    "https://www.bumeran.com.pe/empleos/programador-abap-preencial-experis-peru-1117966739.html",
    "https://www.bumeran.com.pe/empleos/programador-frontend-senior-sonda-del-peru-s.a.-1117973000.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-universidad-de-san-martin-de-porres-1117963157.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-backend-java-inetum-peru-1117961759.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-fullstack-java-js-remoto-protiviti-peru-1117966558.html",
    "https://www.bumeran.com.pe/empleos/desarrollador-frontend-senior-remoto-csti-corp-1117962091.html",
    "https://www.bumeran.com.pe/empleos/frontend-developer-react-js-ts-zutun.-1117969630.html",
    "https://www.bumeran.com.pe/empleos/analista-programador-java-1117972678.html"
]

# === FUNCIÓN PRINCIPAL PARA PROCESAR EN BLOQUES ===
def procesar_en_bloques(urls, tamano_bloque=50):
    total = len(urls)
    bloques = [urls[i:i+tamano_bloque] for i in range(0, total, tamano_bloque)]
    
    for n_bloque, bloque in enumerate(bloques, start=1):
        print(f"\n📦 Procesando bloque {n_bloque} ({len(bloque)} enlaces)...\n")
        resultados = []

        for i, url in enumerate(bloque, start=1):
            print(f"🔍 ({i}) Scrapeando: {url}")
            try:
                html = get_page(url)
                soup = BeautifulSoup(html, 'html.parser')

                titulo_tag = soup.find("h1")
                titulo = titulo_tag.get_text(strip=True) if titulo_tag else "No disponible"

                desc_tag = soup.find("div", class_="jobDescription")
                descripcion = cortar_descripcion(desc_tag.get_text(separator=" ", strip=True)) if desc_tag else "No disponible"

                modalidad = "No especificado"
                for palabra in ["Presencial", "Remoto", "Híbrido"]:
                    if palabra.lower() in html.lower():
                        modalidad = palabra
                        break

                distrito = "No especificado"
                for lugar in ["Lima", "Callao", "Arequipa", "Cusco", "Trujillo"]:
                    if lugar in html:
                        distrito = lugar
                        break

                resultados.append({
                    "URL": url,
                    "Título": titulo,
                    "Modalidad": modalidad,
                    "Distrito / Ubicación": distrito,
                    "Descripción (hasta Beneficios)": descripcion
                })

                time.sleep(random.uniform(2.5, 4.5))

            except Exception as e:
                print(f"⚠️ Error en {url}: {e}")
                resultados.append({
                    "URL": url,
                    "Título": "❌ Error",
                    "Modalidad": "N/A",
                    "Distrito / Ubicación": "N/A",
                    "Descripción (hasta Beneficios)": "N/A"
                })

        df = pd.DataFrame(resultados)
        nombre_archivo = f"scrape_bumeran_bloque_{n_bloque}.xlsx"
        df.to_excel(nombre_archivo, index=False)
        print(f"✅ Bloque {n_bloque} guardado como '{nombre_archivo}'")

# === LLAMADA AL SCRIPT ===
procesar_en_bloques(urls)



📦 Procesando bloque 1 (50 enlaces)...

🔍 (1) Scrapeando: https://www.bumeran.com.pe/empleos/tech-lead-lider-tecnico-frontend-react-nextjs-zutun.-1117959935.html
🔍 (2) Scrapeando: https://www.bumeran.com.pe/empleos/desarrollador-backend-inetum-peru-1117968206.html
🔍 (3) Scrapeando: https://www.bumeran.com.pe/empleos/programador-bms-indra-peru-1117973363.html
🔍 (4) Scrapeando: https://www.bumeran.com.pe/empleos/desarrollador-java-senior-azure-temporal-indra-peru-1117978869.html
🔍 (5) Scrapeando: https://www.bumeran.com.pe/empleos/programador-rpa-rocketbot-uipath-valtx-1117968630.html
🔍 (6) Scrapeando: https://www.bumeran.com.pe/empleos/desarrollador-full-stack-grupo-red-salud-1117971064.html
🔍 (7) Scrapeando: https://www.bumeran.com.pe/empleos/desarrollador-python-odoo-ibr-peru-s.a.-1117961539.html
🔍 (8) Scrapeando: https://www.bumeran.com.pe/empleos/analista-programador-sector-educativo-inlearning-escuelas-de-educacion-superior-1117973244.html
🔍 (9) Scrapeando: https://www.bumeran.com.